In [1]:
from kf import *

In [2]:
A = np.array([[0,1], [0,0]])

In [3]:
B = np.array([[0],[1]])

In [4]:
C = np.array([[0,0]])
D = np.array([[0]])

In [5]:
H = np.array([[0,1]])

In [6]:
Q = np.array([[0.01]])

In [7]:
R = np.array([[1]])

In [8]:
x0 = np.array([[0],[0]])

---

In [9]:
dt = 0.1

In [10]:
t_steps = 100

In [11]:
state_space = signal.StateSpace(A,B,C,D)

In [12]:
time = np.arange(0,t_steps,dt)

In [13]:
F = state_space.to_discrete(dt).A

In [14]:
G = state_space.to_discrete(dt).B

In [15]:
z, x = xtrue(F, G, H, Q, R, x0, t_steps)

---

In [35]:
P0 = np.ones((2,2)) * Q

In [49]:
def covars(F, G, H, Q, R, P0, t_steps):
    '''
    Inputs: F   Xsize*Xsize state transition matrix
         G   Xsize*Vsize state noise transition matrix
         H   Zsize*Xsize observation matrix
         Q   Vsize*Vsize process noise covariance matrix
         R   Zsize*Zsize observation noise covariance matrix
         P0  Xsize*Xsize initial state covariance
         t_steps, number of time-steps to be simulated

     Outputs: W     t_steps*(Xsize*Zsize): Gain history
              Pest  t_steps*(Xsize*Xsize): Estimate Covariance history
              Ppred t_steps*(Xsize*Xsize): Prediction Covariance history
              S     t_steps*(Xsize*Xsize): Innovation Covariance history
    '''
    # First check all matrix dimensions
    Xsize = P0.shape[0]
    Vsize = G.shape[1]
    Zsize = H.shape[0]
    
    assert Xsize == P0.shape[1], "P0 must be square"
    assert F.shape[0] == F.shape[1], "F is non-square"
    assert F.shape[0] == Xsize, "F state dimension does not match Xsize"
    assert G.shape[0] == Xsize, "G does not match dimension of F"
    if len(Q.flatten()) > 1:
        assert Q.shape[0] == Q.shape[1], "Q must be square"
    assert Vsize == Q.shape[1], "Q does not match dimension of G"
    assert H.shape[1] == Xsize, "H and Xsize do not match"
    assert R.shape[0] == R.shape[1], "R must be square"
    assert R.shape[0] == Zsize, "R must match Zsize of H"
    #----------------------------------------------------------------------#
    # End checking of dimensions
    
    # Fix up output matrices 
    W = np.zeros((t_steps, Xsize * Zsize))
    Pest = np.zeros((t_steps, Xsize * Xsize))
    Ppred = np.zeros((t_steps, Xsize * Xsize))
    S = np.zeros((t_steps, Zsize * Zsize))
    
    # initial value
    lPest = P0
    
    # ready to go
    for i in range(0, t_steps - 1):
        # firs the actual calculation in local variables
        lPpred = F @ lPest @ F.T + G @ Q @ G.T
        lS = H @ lPpred @ H.T + R
        lW = lPpred @ H.T/lS
        lPest = lPpred - lW @ lS @ lW.T
        # then record the results in columns of output states
    return lW
    

In [50]:
def xestim(F,G,H,Q,R,x0,P0,z, t_steps):
    ''' 
    Inputs: F   Xsize*Xsize state transition matrix
            G   Xsize*Vsize state noise transition matrix
            H   Zsize*Xsize observation matrix
            Q   Vsize*Vsize process noise covariance matrix
            R   Zsize*Zsize observation noise covariance matrix
            x0  Xsize*1 initial state vector 
            P0  Xsize*Xsize initial state covariance matrix
            z   Zsize*t_steps observation sequence to be filtered

    Outputs: xest  Xsize*t_steps estimated state time history
             xpred Xsize*t_steps predicted state time history
             innov Zsize*t_steps innovation time history
    '''
    assert F.shape[0] == F.shape[1], "F is non-square"
    assert x0.shape[0] == F.shape[0], "X0 does not match dimension of F"
    assert x0.shape[0] == G.shape[0], "G does not match dimension of x0"
    assert Q.shape[0] == Q.shape[1], "Q must be square"
    assert G.shape[1] == Q.shape[0], "Q does not match the dimension of G"
    assert H.shape[1] == x0.shape[0], "H and xsize do not match"
    assert R.shape[0] == R.shape[1], "R must be square"
    assert R.shape[0] == H.shape[0], "R must match Zsize of H"
    assert P0.shape[0] == P0.shape[1], "P0 must be square"
    assert P0.shape[0] == x0.shape[0], "P0 must have dimensions Xsize"
    assert z.shape[0] == H.shape[0], "Observation sequence must have Zsize rows"
    #-----------------#
    # end checking of dimensions
    Xsize = x0.shape[0]
    Zsize = H.shape[0]
   
    # fix up output matrices
    xest = np.zeros((Xsize, t_steps))
    xpred = np.zeros((Xsize, t_steps))
    innov = np.zeros((Zsize, t_steps))
    
    # compute all the necessary gain matrices a priori
    W = covars(F, G, H, Q, R, P0, t_steps)
    
    return W

In [51]:
xestim(F, G, H, Q, R, x0, P0, z, t_steps)

array([[0.06631723],
       [0.009957  ]])

In [52]:
x0.shape

(2, 1)